In [ ]:
import sys
import cv2
import numpy as np
import pyaudio
import wave
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QLabel, QWidget
from PyQt5.QtGui import  QImage, QIcon, QPixmap
from PyQt5.QtCore import QTimer, Qt


class VideoWidget(QWidget):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.video_label = QLabel(self)
        self.video_label.setGeometry(0, 0, 640, 480)

    def set_frame(self, frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        h, w, ch = frame.shape
        bytes_per_line = ch * w
        convert_to_qt_format = QImage(frame.data, w, h, bytes_per_line, QImage.Format_RGB888)
        p = QPixmap.fromImage(convert_to_qt_format).scaled(640, 480, Qt.KeepAspectRatio)
        self.video_label.setPixmap(p)


class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        
        self.setWindowTitle("Webcam and Voice Recorder")
        self.setGeometry(100, 100, 800, 600)

        self.video_widget = VideoWidget(self)
        self.video_widget.setGeometry(50, 50, 640, 480)

        self.capture_button = QPushButton("Capture Frame", self)
        self.capture_button.setGeometry(50, 550, 150, 30)
        self.capture_button.clicked.connect(self.capture_frame)

        self.goto_audio_button = QPushButton("Go to Audio", self)
        self.goto_audio_button.setGeometry(250, 550, 150, 30)
        self.goto_audio_button.clicked.connect(self.switch_to_audio_page)

        self.capture = cv2.VideoCapture(0)
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(10)

    def update_frame(self):
        ret, frame = self.capture.read()
        if ret:
            self.video_widget.set_frame(frame)

    def capture_frame(self):
        ret, frame = self.capture.read()
        if ret:
            cv2.imwrite('captured_frame.jpg', frame)

    def switch_to_audio_page(self):
        self.capture_button.hide()
        self.goto_audio_button.hide()
        self.video_widget.hide()
        audio_widget = AudioWidget(self)
        self.setCentralWidget(audio_widget)


class AudioWidget(QWidget):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.record_button = QPushButton(self)
        self.record_button.setGeometry(250, 200, 150, 150)
        self.record_button.setIcon(QIcon('record_icon.png'))  # Replace 'record_icon.png' with your icon
        self.record_button.setIconSize(self.record_button.size())
        self.record_button.clicked.connect(self.record_voice)

    def record_voice(self):
        audio = pyaudio.PyAudio()
        stream = audio.open(format=pyaudio.paInt16,
                            channels=1,
                            rate=44100,
                            input=True,
                            frames_per_buffer=1024)

        frames = []

        print("recording...")

        for i in range(0, int(44100 / 1024 * 30)):
            data = stream.read(1024)
            frames.append(data)

        stream.stop_stream()
        stream.close()
        audio.terminate()

        wave_file = wave.open('recorded_audio.wav', 'wb')
        wave_file.setnchannels(1)
        wave_file.setsampwidth(audio.get_sample_size(pyaudio.paInt16))
        wave_file.setframerate(44100)
        wave_file.writeframes(b''.join(frames))
        wave_file.close()


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())
